<a href="https://colab.research.google.com/github/DeependraChaddha/RAG_Projects/blob/main/RAG_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##ENVIRONMENT SETUP

In [ ]:
#INSTALLING REQUIRED PACKAGES
!pip install langchain_community tiktoken langchain_openai langchainhub chromadb langchain youtube-transcript-api pytub

In [ ]:
#SETTING LANGSMITH ENVIRONMENT
import os
os.environ["LANGCHAIN_TRACING_V2"]='true'
os.environ["LANGCHAIN_ENDPOINT"]='https://api.smith.langchain.com'
os.environ["LANGCHAIN_API_KEY"]=#YOUR_API_KEY


In [ ]:
#OPENAI API KEY
os.environ["OPENAI_API_KEY"]=#YOUR_API_KEY

##ROUTING

LOGICAL ROUTING

In [ ]:
#MAKING REQUIRED IMPORTS
from typing import Literal
from langchain.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI

#DataModel
class RouteQuery(BaseModel): #Routes user query to most suitable datasource
#datasource attribute/field can only take one of the 3 values
  datasource: Literal["python_docs", "js_docs", "golang_docs"] = Field(#Field function used to add metadata or customize fields of models
      ...,#This is to let PyDantic know that this field cannot be omitted
                                                                      description="Given a user question choose which datasource would be most relevant for answering their question",
    )#This class gives the format in which the llm will answer the query it is provided with in the RAG chain

#LLM with function call
llm=ChatOpenAI(model="gpt-3.5-turbo-0125",temperature=0)
structured_llm=llm.with_structured_output(RouteQuery) #This asks the llm to provide answer in the structure/format specified by class defined above(inherited from BaseModel class of Pydantic)

#Prompt
system="""You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt=ChatPromptTemplate.from_messages([
    ("system",system),
    ("human","{question}"),
    ]
)

#Define Router
router=prompt|structured_llm #This chain takes the prompt and pipes it to the llm which gives an output structured according to RouteQuery

In [ ]:
question="""Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""
#INVOKING THE ROUTER CHAIN
result= router.invoke({"question":question})

In [ ]:
result

In [ ]:
result.datasource

In [ ]:
def choose_route(result):
  if "python_docs" in result.datasource.lower():
    return "chain for python_docs"
  elif "json_docs" in result.datasource.lower():
    return "chain for json_docs"
  else:
    return "golang_docs"

from langchain.runnables import RunnableLambda
#standardize function to be useful in Langchain pipeline architecture
full_chain=router|RunnableLamda(choose_route)

In [ ]:
#INVOKE FULL CHAIN
full_chain.invoke({"question":question})

SEMANTIC ROUTING

In [ ]:
#MAKE ALL REQUIRED IMPORTS
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
#2 PROMPTS